In [289]:
import datetime
import cvxpy as cp

import numpy as np
import pandas as pd

from haversine import haversine_vector, Unit

In [290]:
demand = pd.read_csv('demand.csv',index_col=0)
demand

,subscriber,start_date,start_time,end_date,end_time,trip_duration,car,distance,start_charging_point,end_charging_point,start_station,end_station,start_station_x,start_station_y,end_station_x,end_station_y,start_region,end_region
0,859dcd00,2021-09-23,21:44:45,2021-09-24,00:21:02,02:36:17,17,36,1133,1096,269,279,103.889296,1.396340,103.741013,1.339759,Sengkang,Jurong East
1,328a01df,2021-09-23,22:31:12,2021-09-24,00:58:06,02:26:53,339,90,783,1195,187,282,103.896745,1.370671,103.891881,1.370024,Hougang,Hougang
2,046b1db0,2021-09-23,22:58:07,2021-09-24,00:14:00,01:15:52,150,20,604,770,145,201,103.866182,1.320808,103.941371,1.347284,Kallang,Tampines
3,8992d5fe,2021-09-23,22:59:25,2021-09-24,01:20:07,02:20:42,86,55,495,1207,101,310,103.874236,1.312643,103.923940,1.333285,Kallang,Bedok
4,f78b8c95,2021-09-23,23:01:11,2021-09-24,00:20:18,01:19:07,194,75,356,542,73,122,103.744500,1.375800,103.766838,1.304373,Choa Chu Kang,Clementi
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
362282,b348fb98,2021-12-23,18:59:58,2021-12-23,19:16:59,00:17:00,621,11,689,1108,159,261,103.707724,1.348048,103.764444,1.319651,Jurong West,Clementi
362283,b98ba1dc,2021-12-23,19:00:04,2021-12-23,19:16:39,00:16:35,42,6,330,1464,80,354,103.738434,1.322132,103.713707,1.342613,Jurong East,Jurong West
362284,f86b1b4c,2021-12-23,19:01:55,2021-12-23,19:11:57,00:10:02,23,1,852,1094,196,258,103.885855,1.360295,103.890053,1.362366,Hougang,Hougang
362285,bfce69c1,2021-12-23,19:02:35,2021-12-23,19:15:58,00:13:23,368,3,1545,417,378,110,103.748100,1.381051,103.763191,1.379109,Choa Chu Kang,Bukit Panjang


In [291]:
"""
Temporal clustering 
"""

H = 60 # mins
num_ts = (60*24)/H - 1 #143

def str_to_dt(dt_str):
    return datetime.datetime.strptime(dt_str, '%H:%M:%S').time()

# temporal clustering
demand.insert(3,'start_timeslot', demand['start_time'].apply(lambda x:int(np.floor((60*str_to_dt(x).hour + str_to_dt(x).minute)/H))))
demand.insert(6,'end_timeslot', demand['end_time'].apply(lambda x:int(np.floor((60*str_to_dt(x).hour + str_to_dt(x).minute)/H))))
demand

,subscriber,start_date,start_time,start_timeslot,end_date,end_time,end_timeslot,trip_duration,car,distance,start_charging_point,end_charging_point,start_station,end_station,start_station_x,start_station_y,end_station_x,end_station_y,start_region,end_region
0,859dcd00,2021-09-23,21:44:45,21,2021-09-24,00:21:02,0,02:36:17,17,36,1133,1096,269,279,103.889296,1.396340,103.741013,1.339759,Sengkang,Jurong East
1,328a01df,2021-09-23,22:31:12,22,2021-09-24,00:58:06,0,02:26:53,339,90,783,1195,187,282,103.896745,1.370671,103.891881,1.370024,Hougang,Hougang
2,046b1db0,2021-09-23,22:58:07,22,2021-09-24,00:14:00,0,01:15:52,150,20,604,770,145,201,103.866182,1.320808,103.941371,1.347284,Kallang,Tampines
3,8992d5fe,2021-09-23,22:59:25,22,2021-09-24,01:20:07,1,02:20:42,86,55,495,1207,101,310,103.874236,1.312643,103.923940,1.333285,Kallang,Bedok
4,f78b8c95,2021-09-23,23:01:11,23,2021-09-24,00:20:18,0,01:19:07,194,75,356,542,73,122,103.744500,1.375800,103.766838,1.304373,Choa Chu Kang,Clementi
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
362282,b348fb98,2021-12-23,18:59:58,18,2021-12-23,19:16:59,19,00:17:00,621,11,689,1108,159,261,103.707724,1.348048,103.764444,1.319651,Jurong West,Clementi
362283,b98ba1dc,2021-12-23,19:00:04,19,2021-12-23,19:16:39,19,00:16:35,42,6,330,1464,80,354,103.738434,1.322132,103.713707,1.342613,Jurong East,Jurong West
362284,f86b1b4c,2021-12-23,19:01:55,19,2021-12-23,19:11:57,19,00:10:02,23,1,852,1094,196,258,103.885855,1.360295,103.890053,1.362366,Hougang,Hougang
362285,bfce69c1,2021-12-23,19:02:35,19,2021-12-23,19:15:58,19,00:13:23,368,3,1545,417,378,110,103.748100,1.381051,103.763191,1.379109,Choa Chu Kang,Bukit Panjang


In [292]:
stations = pd.read_csv('stations.csv',index_col=0)
stations

,hrid,name,DSP ID,Administrative ID,kind,address,coords,state,zone,x,y
ID,,,,,,,,,,,
34,angmokio-angmokioav4-blk157,Ang Mo Kio/Ang Mo Kio Av 4/Blk 157,560157-02,SG-560157-002,Parking,"{'city': 'Singapore', 'street': '157 Ang Mo Ki...",SRID=4326;POINT (103.839079 1.3751449),Open,23,103.839079,1.375145
20,angmokio-angmokioave10-blk410,Ang Mo Kio/Ang Mo Kio Ave 10/Blk 410,560410-01,SG-560410-001,Parking,"{'city': 'Singapore', 'street': '410 Ang Mo Ki...",SRID=4326;POINT (103.8546083 1.3632318),Open,23,103.854608,1.363232
298,angmokio-angmokioavenue10-blk449,Ang Mo Kio/Ang Mo Kio Avenue 10/Blk 449,560449-01,SG-560449-001,Parking,"{'city': 'Singapore', 'street': '449 Ang Mo Ki...",SRID=4326;POINT (103.8558 1.36805),Open,23,103.855800,1.368050
140,angmokio-angmokioavenue10-blk544,Ang Mo Kio/Ang Mo Kio Avenue 10/Blk 544,560540-01,SG-560540-001,Parking,"{'city': 'Singapore', 'street': '540/546 Ang M...",SRID=4326;POINT (103.856048 1.373765),Open,23,103.856048,1.373765
329,angmokio-angmokioavenue10-blk559,Ang Mo Kio/Ang Mo Kio Avenue 10/Blk 559,560559-01,SG-560559-001,Parking,"{'city': 'Singapore', 'street': '559 Ang Mo Ki...",SRID=4326;POINT (103.857277 1.371022),Open,23,103.857277,1.371022
...,...,...,...,...,...,...,...,...,...,...,...
374,yishun-yishunstreet41-blk456a-deck1b,Yishun/Yishun Street 41/Blk 456A/Deck 1B,761456-01,SG-761456-001,Parking,"{'city': 'Singapore', 'street': '456A Yishun S...",SRID=4326;POINT (103.8451 1.419846),Open,27,103.845100,1.419846
238,yishun-yishunstreet51-blk501-deck1a,Yishun/Yishun Street 51/Blk 501/Deck 1A,760501-01,SG-760501-001,Parking,"{'city': 'Singapore', 'street': '501 Yishun St...",SRID=4326;POINT (103.840604 1.416572),Open,27,103.840604,1.416572
63,yishun-yishunstreet51-blk504-deck2,Yishun/Yishun Street 51/Blk 504/Deck 2,760504-01,SG-760504-001,Parking,"{'city': 'Singapore', 'street': '504 Yishun st...",SRID=4326;POINT (103.844755 1.418416),Open,27,103.844755,1.418416


In [293]:
stn_coords = list(zip(stations.y, stations.x))
#stn_coords

In [294]:
"""
Spatial clustering
"""

d = 0.5 # km radius from cluster center which is a station.
#station such that it has has most number of stations within a radius from it (cluster center ). 

D = haversine_vector(stn_coords, stn_coords, Unit.KILOMETERS, comb=True)
R = 1*(D < d) # 1 km threshold

# ilp to select cluster centers
x = cp.Variable((len(stations),),boolean=True)
prob = cp.Problem(cp.Minimize(cp.sum(x)), [x.T@R>=1])
prob.solve()

# map each station to a cluster
stations['cc'] = x.value # x tells whether a station is a cluster center or not.

In [295]:
stations

cluster_centers = stations.loc[stations.cc==1]
cc_coords = list(zip(cluster_centers.y, cluster_centers.x))
stations['cc_id'] = np.argmin(haversine_vector(cc_coords, stn_coords, Unit.KILOMETERS, comb=True), axis=1)

In [296]:

station_to_cluster_map = dict(zip(stations.index, stations.cc_id))
# stations.index is actually the ID in staitons.csv, stations.cc_id is the true index of the stations.csv file. 

In [297]:
# for x in station_to_cluster_map.keys(): # find station id given cluster id like 165.
#     if (station_to_cluster_map[x]==165):
#         print(x)

184
350
293
130


In [298]:

demand['start_cluster'] = demand['start_station'].apply(lambda x: station_to_cluster_map[x])
demand['end_cluster'] = demand['end_station'].apply(lambda x: station_to_cluster_map[x])


In [300]:
stations.to_csv(f'station_cc_id_{H}_{d}.csv', index=False)

In [302]:
# df = pd.DataFrame(columns=['st_dt','st_ts','car','clstr','stn'] )
# df.loc[0] = {'st_dt':1,'st_ts':0,'car':4,'clstr':'h','stn':'h1'}
# df.loc[1] = {'st_dt':1,'st_ts':0,'car':3,'clstr':'f','stn':'f1'}
# df.loc[2] = {'st_dt':1,'st_ts':0,'car':4,'clstr':'f','stn':'f2'}
# df.loc[3] = {'st_dt':1,'st_ts':1,'car':2,'clstr':'h','stn':'h2'}
# df.loc[4] = {'st_dt':1,'st_ts':1,'car':1,'clstr':'f','stn':'f3'}
# df.loc[5] = {'st_dt':1,'st_ts':1,'car':1,'clstr':'f','stn':'f2'}
# df.loc[6] = {'st_dt':1,'st_ts':2,'car':6,'clstr':'g','stn':'g2'}
# df

# dfpv = pd.pivot_table(df,values='car',index=["st_dt","st_ts"], columns=["clstr",], aggfunc=lambda x: len(x.unique()))  # experiment to check len(x.unique()) vs. len(x)
# dfpv.fillna(0, inplace=True)
# dfpv

,st_dt,st_ts,car,clstr,stn
0,1,0,4,h,h1
1,1,0,3,f,f1
2,1,0,4,f,f2
3,1,1,2,h,h2
4,1,1,1,f,f3
5,1,1,1,f,f2
6,1,2,6,g,g2


In [304]:
OU = pd.pivot_table(demand, values = 'car', index=["start_date","start_timeslot"], columns=["start_cluster",], aggfunc=lambda x: len(x.unique())) #x.unique()
OU.fillna(0, inplace=True)

#start_station
# missing_clusters = [x for x in range(OU.columns[-1]) if x not in OU.columns]
# missing_clusters.sort()

# for ms in missing_clusters:
#     OU.insert(ms, ms, 0)

In [305]:
OU.insert(1,"idx",range(len(OU))) # ADD index to identify missing timeslot
OU.head(10)

"""
remove the first date with a few timeslot to ensure that 10 min and 60 mins start on the same day. they are alinged. 
TODO:
"""


start_cluster                0 idx    1         2              3    4    5  \
start_station               20      298  329   34  272  285  286  203  195   
start_date start_timeslot                                                    
2021-09-23 21              0.0   0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
           22              0.0   1  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
           23              0.0   2  0.0  0.0  1.0  0.0  0.0  0.0  1.0  0.0   
2021-09-24 0               0.0   3  0.0  1.0  1.0  0.0  0.0  1.0  0.0  0.0   
           1               2.0   4  0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0   
           2               0.0   5  0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0   
           3               0.0   6  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
           4               1.0   7  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0   
           5               0.0   8  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
           6               1.0   9  0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0   
           7               0.0  10  0.0  2.0  0.0  0.0  0.0  1.0  0.0  0.0   
           8               0.0  11  0.0  0.0  0.0  0.0  0.0  0.0  0.0  2.0   
           9               0.0  12  0.0  0.0  0.0  0.0  0.0  0.0  0.0  2.0   
           10              0.0  13  0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0   
           11              0.0  14  0.0  1.0  0.0  0.0  0.0  0.0  0.0  1.0   
           12              1.0  15  0.0  3.0  0.0  0.0  0.0  0.0  1.0  0.0   
           13              3.0  16  3.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0   
           14              0.0  17  1.0  0.0  0.0  0.0  0.0  0.0  1.0  1.0   
           15              1.0  18  0.0  1.0  0.0  0.0  0.0  0.0  1.0  1.0   
           16              0.0  19  0.0  0.0  0.0  0.0  0.0  1.0  3.0  3.0   

start_cluster              ...  190  191  192                 193            \
start_station              ...  164   21   63  237  349  374  107  238  337   
start_date start_timeslot  ...                                                
2021-09-23 21              ...  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
           22              ...  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
           23              ...  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
2021-09-24 0               ...  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
           1               ...  0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0   
           2               ...  0.0  0.0  0.0  1.0  0.0  0.0  1.0  0.0  0.0   
           3               ...  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
           4               ...  0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0   
           5               ...  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
           6               ...  0.0  1.0  2.0  0.0  2.0  1.0  1.0  1.0  1.0   
           7               ...  0.0  1.0  1.0  0.0  0.0  1.0  0.0  0.0  0.0   
           8               ...  0.0  0.0  1.0  0.0  0.0  1.0  0.0  1.0  0.0   
           9               ...  0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0   
           10              ...  1.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0  1.0   
           11              ...  1.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0   
           12              ...  0.0  0.0  0.0  1.0  0.0  0.0  2.0  0.0  1.0   
           13              ...  0.0  1.0  2.0  1.0  0.0  0.0  1.0  0.0  0.0   
           14              ...  0.0  0.0  1.0  0.0  0.0  0.0  0.0  3.0  0.0   
           15              ...  0.0  0.0  2.0  3.0  0.0  0.0  0.0  1.0  0.0   
           16              ...  1.0  0.0  0.0  0.0  0.0  0.0  0.0  2.0  0.0   

start_cluster                   
start_station              375  
start_date start_timeslot       
2021-09-23 21              0.0  
           22              0.0  
           23              0.0  
2021-09-24 0               0.0  
           1               0.0  
           2               0.0  
           3               0.0  
           4               0.0  
           5            

In [306]:
OU.tail(10)

start_cluster                0   idx    1         2              3    4    5  \
start_station               20        298  329   34  272  285  286  203  195   
start_date start_timeslot                                                      
2021-12-23 19              0.0  2182  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   

start_cluster              ...  190  191  192                 193            \
start_station              ...  164   21   63  237  349  374  107  238  337   
start_date start_timeslot  ...                                                
2021-12-23 19              ...  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   

start_cluster                   
start_station              375  
start_date start_timeslot       
2021-12-23 19              0.0  

[1 rows x 383 columns]

In [308]:
miss_idx_amt = {} # missing index and amount

for r in range(len(OU)-1):
    
    f_ts = OU.index[r][1] # 1st time slot
    s_ts = OU.index[r+1][1] # 2nd time slot

    if(s_ts - f_ts > 1 ):
        miss_idx_amt[OU.idx[r]] = s_ts - f_ts - 1

    elif(s_ts - f_ts < 1):
        if (num_ts - f_ts > 0): miss_idx_amt[OU.idx[r]] = num_ts - f_ts
        if (s_ts != 0): miss_idx_amt[OU.idx[r+1]] = -1*(s_ts+1)




In [309]:
OU.head(10)

start_cluster                0 idx    1         2              3    4    5  \
start_station               20      298  329   34  272  285  286  203  195   
start_date start_timeslot                                                    
2021-09-23 21              0.0   0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
           22              0.0   1  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
           23              0.0   2  0.0  0.0  1.0  0.0  0.0  0.0  1.0  0.0   
2021-09-24 0               0.0   3  0.0  1.0  1.0  0.0  0.0  1.0  0.0  0.0   
           1               2.0   4  0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0   
           2               0.0   5  0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0   
           3               0.0   6  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
           4               1.0   7  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0   
           5               0.0   8  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
           6               1.0   9  0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0   
           7               0.0  10  0.0  2.0  0.0  0.0  0.0  1.0  0.0  0.0   
           8               0.0  11  0.0  0.0  0.0  0.0  0.0  0.0  0.0  2.0   
           9               0.0  12  0.0  0.0  0.0  0.0  0.0  0.0  0.0  2.0   
           10              0.0  13  0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0   
           11              0.0  14  0.0  1.0  0.0  0.0  0.0  0.0  0.0  1.0   
           12              1.0  15  0.0  3.0  0.0  0.0  0.0  0.0  1.0  0.0   
           13              3.0  16  3.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0   
           14              0.0  17  1.0  0.0  0.0  0.0  0.0  0.0  1.0  1.0   
           15              1.0  18  0.0  1.0  0.0  0.0  0.0  0.0  1.0  1.0   
           16              0.0  19  0.0  0.0  0.0  0.0  0.0  1.0  3.0  3.0   
           17              2.0  20  1.0  0.0  0.0  1.0  0.0  1.0  1.0  1.0   
           18              2.0  21  0.0  0.0  0.0  2.0  2.0  1.0  1.0  0.0   
           19              1.0  22  2.0  0.0  0.0  2.0  0.0  0.0  1.0  0.0   
           20              0.0  23  1.0  1.0  0.0  0.0  2.0  3.0  1.0  2.0   
           21              0.0  24  0.0  0.0  0.0  1.0  3.0  2.0  1.0  0.0   
           22              2.0  25  2.0  0.0  0.0  1.0  0.0  0.0  1.0  1.0   
           23              2.0  26  0.0  2.0  0.0  0.0  0.0  0.0  1.0  0.0   
2021-09-25 0               0.0  27  0.0  0.0  1.0  1.0  0.0  1.0  0.0  1.0   
           1               2.0  28  1.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0   
           2               1.0  29  0.0  0.0  0.0  0.0  0.0  0.0  0.0  2.0   
           3               1.0  30  1.0  2.0  0.0  0.0  0.0  0.0  0.0  1.0   
           4               0.0  31  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
           5               0.0  32  0.0  1.0  0.0  0.0  1.0  0.0  0.0  0.0   
           6               1.0  33  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
           7               1.0  34  0.0  0.0  0.0  0.0  0.0  1.0  0.0  1.0   
           8               0.0  35  0.0  0.0  1.0  0.0  0.0  1.0  0.0  1.0   
           9               1.0  36  0.0  0.0  1.0  1.0  0.0  1.0  1.0  2.0   
           10              1.0  37  0.0  1.0  0.0  1.0  0.0  0.0  0.0  0.0   
           11              0.0  38  0.0  1.0  0.0  0.0  0.0  2.0  0.0  0.0   
           12              2.0  39  0.0  0.0  1.0  0.0  0.0  2.0  2.0  2.0   
           13              1.0  40  0.0  3.0  1.0  0.0  0.0  0.0  4.0  0.0   
           14              0.0  41  1.0  2.0  0.0  0.0  0.0  1.0  2.0  0.0   
           15              1.0  42  0.0  1.0  0.0  0.0  0.0  0.0  1.0  0.0   
           16              2.0  43  0.0  2.0  0.0  0.0  0.0  1.0  0.0  1.0   
           17              2.0  44  1.0  2.0  0.0  0.0  0.0  0.0  0.0  0.0   
           18              2.0  45  0.0  0.0  0.0  1.0  0.0  1.0  1.0  1.0   
           19              0.0  46  0.0  0.0  0.0  1.0  0.0  2.0  2.0  1.0   
           20              1.0  47  1.0  1.0  0.0  1.0  1.0  1.0  1.0  0.0   
           21         

In [310]:
OU.to_csv(f'outflow_station_{H}_{d}.csv', index=False)


In [311]:
new_df = pd.read_csv("outflow_station_10_0.5.csv")
new_df

idx_shift=0

for i in list(miss_idx_amt.keys()):
    amt = miss_idx_amt[i]
    inc = int(amt/abs(amt))
    if (inc >= 0):
        strt = 0
    else:
        strt = -1
    for j in range(strt,amt,inc):
        
        new_df.loc[1+i+j+0.5+idx_shift] = 0
        new_df = new_df.sort_index().reset_index(drop=True)
        idx_shift+=1

new_df.to_csv("outflow_station_10_0.5.csv")
new_df


,Unnamed: 0,Unnamed: 0.1,0,idx,1,1.1,2,2.1,2.2,3,...,190,191,192,192.1,192.2,192.3,193,193.1,193.2,193.3
0,0,0,20.0,NaN,298.0,329.0,34.0,272.0,285.0,286.0,...,164.0,21.0,63.0,237.0,349.0,374.0,107.0,238.0,337.0,375.0
1,1,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3,3,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13085,13085,13085,0.0,13027.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
13086,13086,13086,0.0,13028.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
13087,13087,13087,0.0,13029.0,0.0,0.0,1.0,1.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0
13088,13088,13088,0.0,13030.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [312]:
IN = pd.pivot_table(demand, values = 'car', index=["end_date","end_timeslot"], columns=["end_cluster"], aggfunc=lambda x: len(x)) #x.unique()
IN.fillna(0, inplace=True)

#end_station
# missing_clusters = [x for x in range(IN.columns[-1]) if x not in IN.columns]
# missing_clusters.sort()

# for ms in missing_clusters:
#     IN.insert(ms, ms, 0)

In [313]:
IN.head(5)

end_cluster              0    1         2              3    4    5         \
end_station              20   298  329  34   272  285  286  203  195  294   
end_date   end_timeslot                                                     
2021-09-24 0             0.0  1.0  1.0  0.0  0.0  0.0  2.0  0.0  0.0  0.0   
           1             0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0   
           2             1.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0   
           3             1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
           4             0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   

end_cluster              ...  190  191  192                 193                 
end_station              ...  164  21   63   237  349  374  107  238  337  375  
end_date   end_timeslot  ...                                                    
2021-09-24 0             ...  0.0  1.0  0.0  1.0  1.0  0.0  0.0  1.0  0.0  0.0  
           1             ...  0.0  1.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0  
           2             ...  0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0  
           3             ...  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
           4             ...  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  

[5 rows x 382 columns]

In [314]:
IN.tail(5)

end_cluster              0    1         2              3    4    5         \
end_station              20   298  329  34   272  285  286  203  195  294   
end_date   end_timeslot                                                     
2021-12-23 15            0.0  2.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  1.0   
           16            0.0  0.0  0.0  0.0  0.0  0.0  1.0  3.0  1.0  0.0   
           17            2.0  1.0  0.0  1.0  1.0  0.0  0.0  1.0  0.0  2.0   
           18            0.0  0.0  2.0  0.0  0.0  0.0  2.0  1.0  1.0  0.0   
           19            2.0  1.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0   

end_cluster              ...  190  191  192                 193                 
end_station              ...  164  21   63   237  349  374  107  238  337  375  
end_date   end_timeslot  ...                                                    
2021-12-23 15            ...  2.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0  1.0  
           16            ...  0.0  1.0  1.0  0.0  2.0  1.0  1.0  0.0  0.0  0.0  
           17            ...  0.0  1.0  1.0  1.0  2.0  0.0  1.0  1.0  0.0  2.0  
           18            ...  1.0  0.0  2.0  2.0  1.0  0.0  1.0  2.0  0.0  2.0  
           19            ...  0.0  0.0  1.0  1.0  1.0  0.0  0.0  0.0  0.0  0.0  

[5 rows x 382 columns]

In [315]:
IN.insert(1,"idx",range(len(IN))) # ADD index to identify missing timeslot

In [316]:
miss_idx_amt = {} # missing index and amount

for r in range(len(IN)-1):
    
    f_ts = IN.index[r][1] # 1st time slot
    s_ts = IN.index[r+1][1] # 2nd time slot

    if(s_ts - f_ts > 1 ):
        miss_idx_amt[IN.idx[r]] = s_ts - f_ts - 1

    elif(s_ts - f_ts < 1):
        if (num_ts - f_ts > 0): miss_idx_amt[IN.idx[r]] = num_ts - f_ts
        if (s_ts != 0): miss_idx_amt[IN.idx[r+1]] = -1*(s_ts+1)



In [317]:
IN.to_csv(f'inflow_station_{H}_{d}.csv', index=False)

In [318]:
new_df = pd.read_csv("inflow_station_10_0.5.csv")
new_df

idx_shift=0

for i in list(miss_idx_amt.keys()):
    amt = miss_idx_amt[i]
    inc = int(amt/abs(amt))
    if (inc >= 0):
        strt = 0
    else:
        strt = -1
    for j in range(strt,amt,inc):
        
        new_df.loc[1+i+j+0.5+idx_shift] = 0
        new_df = new_df.sort_index().reset_index(drop=True)
        idx_shift+=1

new_df.to_csv("inflow_station_10_0.5.csv")
new_df


,Unnamed: 0,Unnamed: 0.1,0,idx,1,1.1,2,2.1,2.2,3,...,190,191,192,192.1,192.2,192.3,193,193.1,193.2,193.3
0,0,0,20.0,NaN,298.0,329.0,34.0,272.0,285.0,286.0,...,164.0,21.0,63.0,237.0,349.0,374.0,107.0,238.0,337.0,375.0
1,1,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
2,2,2,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3,3,0.0,2.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4,4,0.0,3.0,1.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13073,13073,13073,0.0,13024.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
13074,13074,13074,0.0,13025.0,0.0,2.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
13075,13075,13075,2.0,13026.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
13076,13076,13076,0.0,13027.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
